# Se connecter à la base de données

Et sélectionner les éléments uniques des produits

In [1]:
import sqlite3

# Se connecter à la base de données
conn = sqlite3.connect('inventory.db')
cursor = conn.cursor()

# Récupérer les noms d'articles uniques
cursor.execute("SELECT DISTINCT item_name FROM products")
items = [row[0] for row in cursor.fetchall()]

# Importation de genai

In [ ]:
import google.generativeai as genai
genai.configure(api_key='YOUR_API_KEY')

# Pour récupérer les demandes de <br>l'utilisateur à partir de la prompt<br>
et créer une requête SQL

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-flash')

# Demande de l'utilisateur
user_query = "Combien de jean sont disponible dans le magazin?"

# Interroger le modèle pour générer une clause WHERE SQL
prompt = f"""
Étant donné la demande de l'utilisateur : "{user_query}", générez une clause SQL pour filtrer la table 
products et sélectionner ce que demande l'utilisateur.
Voici les colonnes disponibles : item_name, quantity, price, color

Voici la liste de tous les articles disponibles dans le magasin : {items}

Répondez uniquement avec le code SQL si l'article est disponible dans le magasin. Par exemple, 
voici un code : SELECT DISTINCT item_name FROM products.
Si la demande de l'utilisateur ne correspond à aucun article disponible dans le magasin, vous 
devez retourner No_result_found sans requet sql svp
"""

response = model.generate_content(prompt)

In [35]:
response.text

"```sql\nSELECT SUM(quantity) AS total_jeans FROM products WHERE item_name = 'Jeans';\n```\n"

# Nettoyage de la sortie

In [36]:
def extract_sql_query(text):
    # Supprimer les espaces blancs au début et à la fin
    text = text.strip()
    
    # Supprimer les backticks des blocs de code si présents
    if text.startswith('```sql') and text.endswith('```'):
        text = text.strip('```').replace('sql\n', '').strip()
    
    return text

# Exemple d'utilisation

sql_query = extract_sql_query(response.text)
print(sql_query)

SELECT SUM(quantity) AS total_jeans FROM products WHERE item_name = 'Jeans';


# Filtrage de la base de données

In [37]:
conn = sqlite3.connect('inventory.db')
cursor = conn.cursor()

cursor.execute(sql_query)
result = [row[0] for row in cursor.fetchall()]

In [38]:
result

[130]

# Demande finale au LLM avec les données filtrées

In [ ]:
# Interroger le modèle pour générer une clause WHERE SQL
prompt = f"""
Vous êtes un assistant de magasin, veuillez répondre à l'utilisateur sur la 
base des documents récupérés
ces documents ont été récupérés de la base de données et ils contiennent les informations 
sur ce que l'utilisateur a demandé.
Voici la demande originale de l'utilisateur : {user_query}
et voici les documents récupérés de la base de données : {result}
si le résultat n'est pas vide, cela confirme que la demande de l'utilisateur est disponible
"""

response = model.generate_content(prompt)

In [40]:
response.text

'Bonjour !  Nous avons actuellement 130 jeans en stock.\n'